<a href="https://colab.research.google.com/github/PRAGYAMISHRA04/Time-series-analysis/blob/main/Crypto_Research_Wavenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import tensorflow as tf
#import os

#tpu = None
#try: # detect TPUs
#    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
#    print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
#    tf.config.experimental_connect_to_cluster(tpu)
#    tf.tpu.experimental.initialize_tpu_system(tpu)
#    strategy = tf.distribute.experimental.TPUStrategy(tpu)
#    print("REPLICAS: ", strategy.num_replicas_in_sync)
#except ValueError: # detect GPUs
#    strategy = tf.distribute.MirroredStrategy() # for GPU or multi-GPU machines

#print("Number of accelerators: ", strategy.num_replicas_in_sync)

#resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
#tf.config.experimental_connect_to_cluster(resolver)
#tf.tpu.experimental.initialize_tpu_system(resolver)

#**Importing Libraries**

In [ ]:
!pip install tensorflow-addons
import tensorflow as tf
import os
import tensorflow_addons as tfa
"""! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download g-research-crypto-forecasting"""
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

     |████████████████████████████████| 1.1 MB 15.1 MB/s 
Mounted at /content/drive


In [ ]:
! unzip "/content/drive/MyDrive/Crypto-research/train.csv (1).zip" #"/content/drive/MyDrive/TRAIN.csv.zip"

Archive:  /content/drive/MyDrive/Crypto-research/train.csv (1).zip
  inflating: train.csv               


In [ ]:
from tensorflow.keras.layers import Concatenate, Add, GRU, GlobalAveragePooling2D, Dense, Dropout, InputLayer, Input , Bidirectional, LSTM, Conv1D, Multiply, Masking
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Activation
from tensorflow.keras.metrics import RootMeanSquaredError, MeanAbsolutePercentageError , MeanSquaredError
from keras.utils.generic_utils import get_custom_objects
import math as m
import time
import gc
import os

In [ ]:
def act_bipolar(x):
    '''Bipolar'''
    return ((1-tf.math.exp(-x)) / (1+tf.math.exp(-x)))
def act_SiLU(x):
    '''SiLU'''
    return (x/(1+tf.math.exp(-x)))
def act_softplus(x):
    '''Softplus'''
    return tf.nn.softplus(x)

def act_LRELU(x):
  '''Leaky RELU'''
  return tf.nn.leaky_relu(x)

def act_swish(x):
   '''Swish'''
   return x * tf.sigmoid(x)

def act_Mcubic(x):
    '''Monotonic Cubic'''
    return (tf.math.pow(x,3) + x)

def act_NMcubic(x):
    '''Non Monotonic Cubic'''
    return (x - tf.math.pow(x,3))

def act_DSU(x):
    '''Decaying Sine Unit'''
    return (tf.constant(m.pi)/2)*(tf.experimental.numpy.sinc(x-tf.constant(m.pi)) - tf.experimental.numpy.sinc(x+tf.constant(m.pi))) 

def act_GCU(x):
    '''Growing Cosine Unit'''
    return x*tf.cos(x)

def act_shiftedSinc(x):
    '''Shifted Sinc'''
    return tf.constant(m.pi)*tf.experimental.numpy.sinc(x-tf.constant(m.pi))

def act_z2cosz(x):
  ''' x2cos(x)'''
  return x*x*tf.cos(x)

def act_sinz(x):
  ''' sin(x)'''
  return tf.sin(x)

def act_Mish(x):
  '''Mish'''
  return x*tf.tanh(tf.math.log(1+tf.math.exp(x)))

def act_GELU(x):
  '''GELU'''
  return tf.keras.activations.gelu(x)

def act_Shifted_Quadratic(x):
  '''Shifted Quadratic'''
  return (tf.math.pow(x,2) + x)
  
#get_custom_objects().update({'bipolar': Activation(act_bipolar)})
#get_custom_objects().update({'silu': Activation(act_SiLU)})
#get_custom_objects().update({'softplus': Activation(act_softplus)})
get_custom_objects().update({'lrelu': Activation(act_LRELU)})
#get_custom_objects().update({'swish': Activation(act_swish)})
get_custom_objects().update({'mcubic': Activation(act_Mcubic)})
get_custom_objects().update({'nmcubic': Activation(act_NMcubic)})
get_custom_objects().update({'dsu': Activation(act_DSU)})
get_custom_objects().update({'gcu': act_GCU})
get_custom_objects().update({'shiftedsinc': Activation(act_shiftedSinc)})
#get_custom_objects().update({'mish': Activation(act_Mish)})
#get_custom_objects().update({'gelu': Activation(act_GELU)})
get_custom_objects().update({'shiftedquadratic': Activation(act_Shifted_Quadratic)})
get_custom_objects().update({'z2cosz': Activation(act_z2cosz)})
get_custom_objects().update({'sin': Activation(act_sinz)})
class TimeCalcCallback(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_time_start = time.time()    
    def on_epoch_end(self, epoch, logs=None):
        times.append(time.time() - self.epoch_time_start)

#**Importing Data**

In [ ]:
dataset=pd.read_csv("train.csv")
dataset.head()

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
0,1514764860,2,40.0,2376.5800,2399.5000,2357.1400,2374.5900,19.233005,2373.116392,-0.004218
1,1514764860,0,5.0,8.5300,8.5300,8.5300,8.5300,78.380000,8.530000,-0.014399
2,1514764860,1,229.0,13835.1940,14013.8000,13666.1100,13850.1760,31.550062,13827.062093,-0.014643
3,1514764860,5,32.0,7.6596,7.6596,7.6567,7.6576,6626.713370,7.657713,-0.013922
4,1514764860,7,5.0,25.9200,25.9200,25.8740,25.8770,121.087310,25.891363,-0.008264


#**Data Preprocessing**

In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24236806 entries, 0 to 24236805
Data columns (total 10 columns):
 #   Column     Dtype  
---  ------     -----  
 0   timestamp  int64  
 1   Asset_ID   int64  
 2   Count      float64
 3   Open       float64
 4   High       float64
 5   Low        float64
 6   Close      float64
 7   Volume     float64
 8   VWAP       float64
 9   Target     float64
dtypes: float64(8), int64(2)
memory usage: 1.8 GB


In [ ]:
dataset.isnull().sum()

timestamp         0
Asset_ID          0
Count             0
Open              0
High              0
Low               0
Close             0
Volume            0
VWAP              9
Target       750338
dtype: int64

In [ ]:
dataset=dataset.dropna()

In [ ]:
df=dataset.groupby("Asset_ID")
bitcoin_data=df.get_group(1)
bitcoin_data.shape

(1955978, 10)

In [ ]:
bitcoin_data=bitcoin_data.drop(['Asset_ID','timestamp'],axis=1)
bitcoin_data.head()
#data=bitcoin_data['VWAP']

,Count,Open,High,Low,Close,Volume,VWAP,Target
2,229.0,13835.194,14013.8,13666.11,13850.176,31.550062,13827.062093,-0.014643
10,235.0,13835.036,14052.3,13680.00,13828.102,31.046432,13840.362591,-0.015037
18,528.0,13823.900,14000.4,13601.00,13801.314,55.061820,13806.068014,-0.010309
26,435.0,13802.512,13999.0,13576.28,13768.040,38.780529,13783.598101,-0.008999
34,742.0,13766.000,13955.9,13554.44,13724.914,108.501637,13735.586842,-0.008079


In [ ]:
def upper_shadow(df): return df['High'] - np.maximum(df['Close'], df['Open'])
def lower_shadow(df): return np.minimum(df['Close'], df['Open']) - df['Low']

def get_features(df, row = False):
    df_feat = df
    df_feat['spread'] = df_feat['High'] - df_feat['Low']
    df_feat['mean_trade'] = df_feat['Volume']/df_feat['Count']
    df_feat['log_price_change'] = np.log(df_feat['Close']/df_feat['Open'])
    df_feat['upper_Shadow'] = upper_shadow(df_feat)
    df_feat['lower_Shadow'] = lower_shadow(df_feat)
    df_feat["high_div_low"] = df_feat["High"] / df_feat["Low"]
    df_feat['trade'] = df_feat['Close'] - df_feat['Open']
    df_feat['gtrade'] = df_feat['trade'] / df_feat['Count']
    df_feat['shadow1'] = df_feat['trade'] / df_feat['Volume']
    df_feat['shadow3'] = df_feat['upper_Shadow'] / df_feat['Volume']
    df_feat['shadow5'] = df_feat['lower_Shadow'] / df_feat['Volume']
    df_feat['diff1'] = df_feat['Volume'] - df_feat['Count']
    df_feat['mean1'] = (df_feat['shadow5'] + df_feat['shadow3']) / 2
    df_feat['mean2'] = (df_feat['shadow1'] + df_feat['Volume']) / 2
    df_feat['mean3'] = (df_feat['trade'] + df_feat['gtrade']) / 2
    df_feat['mean4'] = (df_feat['diff1'] + df_feat['upper_Shadow']) / 2
    df_feat['mean5'] = (df_feat['diff1'] + df_feat['lower_Shadow']) / 2
    df_feat['UPS'] = (df_feat['High'] - np.maximum(df_feat['Close'], df_feat['Open']))
    df_feat['UPS'] = df_feat['UPS']
    df_feat['LOS'] = (np.minimum(df_feat['Close'], df_feat['Open']) - df_feat['Low'])
    df_feat['LOS'] = df_feat['LOS']
    df_feat['RNG'] = ((df_feat['High'] - df_feat['Low']) / df_feat['VWAP'])
    df_feat['RNG'] = df_feat['RNG']
    df_feat['MOV'] = ((df_feat['Close'] - df_feat['Open']) / df_feat['VWAP'])
    df_feat['MOV'] = df_feat['MOV']
    df_feat['CLS'] = ((df_feat['Close'] - df_feat['VWAP']) / df_feat['VWAP'])
    df_feat['CLS'] = df_feat['CLS']
    df_feat['LOGVOL'] = np.log(1. + df_feat['Volume'])
    df_feat['LOGVOL'] = df_feat['LOGVOL']
    df_feat['LOGCNT'] = np.log(1. + df_feat['Count'])
    df_feat['LOGCNT'] = df_feat['LOGCNT']
    df_feat["Close/Open"] = df_feat["Close"] / df_feat["Open"]
    df_feat["Close-Open"] = df_feat["Close"] - df_feat["Open"]
    df_feat["High-Low"] = df_feat["High"] - df_feat["Low"]
    df_feat["High/Low"] = df_feat["High"] / df_feat["Low"]
    if row: df_feat['Mean'] = df_feat[['Open', 'High', 'Low', 'Close']].mean()
    else: df_feat['Mean'] = df_feat[['Open', 'High', 'Low', 'Close']].mean(axis = 1)
    df_feat["High/Mean"] = df_feat["High"] / df_feat["Mean"]
    df_feat["Low/Mean"] = df_feat["Low"] / df_feat["Mean"]
    df_feat["Volume/Count"] = df_feat["Volume"] / (df_feat["Count"] + 1)
    mean_price = df_feat[['Open', 'High', 'Low', 'Close']].mean(axis=1)
    median_price = df_feat[['Open', 'High', 'Low', 'Close']].median(axis=1)
    df_feat['high2mean'] = df_feat['High'] / mean_price
    df_feat['low2mean'] = df_feat['Low'] / mean_price
    df_feat['high2median'] = df_feat['High'] / median_price
    df_feat['low2median'] = df_feat['Low'] / median_price
    df_feat['volume2count'] = df_feat['Volume'] / (df_feat['Count'] + 1)
    return df_feat

In [ ]:
CASE=2 # CASE =1 ,2 ,3 
FEATURES=list()

if CASE==1:
  FEATURES.append('VWAP')
elif CASE==2:
  FEATURES=['Count','Open','High','Low','Close','Volume','VWAP','Target']
else :
  bitcoin_data=get_features(bitcoin_data)
  FEATURES=list(bitcoin_data.columns)

data=bitcoin_data[FEATURES]
data.head()

,Count,Open,High,Low,Close,Volume,VWAP,Target
2,229.0,13835.194,14013.8,13666.11,13850.176,31.550062,13827.062093,-0.014643
10,235.0,13835.036,14052.3,13680.00,13828.102,31.046432,13840.362591,-0.015037
18,528.0,13823.900,14000.4,13601.00,13801.314,55.061820,13806.068014,-0.010309
26,435.0,13802.512,13999.0,13576.28,13768.040,38.780529,13783.598101,-0.008999
34,742.0,13766.000,13955.9,13554.44,13724.914,108.501637,13735.586842,-0.008079


In [ ]:
Scaler=MinMaxScaler()
data=Scaler.fit_transform(data)

In [ ]:
size=data.shape[0]
train_size=m.floor(0.8*size)
val_size=m.floor(0.1*size)
train_data= data[: train_size] 
val_data = data[train_size: train_size+val_size ]
test_data = data[train_size+val_size:]

In [ ]:
print('train:',train_data.shape)
print('valid:',val_data.shape)
print('test:',test_data.shape)

train: (1564782, 8)
valid: (195597, 8)
test: (195599, 8)


In [ ]:
def create_dataset(df, time_step=1):
    dataX, dataY = [], []
    for i in range(len(df)-time_step-1):
        a = df[i:(i+time_step)]   
        dataX.append(a)
        dataY.append(df[i + time_step])
        if i==15646:
          break
    return np.array(dataX), np.array(dataY)

In [ ]:
train_X,train_Y = create_dataset(np.array(train_data),100)
val_X,val_Y=create_dataset(np.array(val_data),100)
test_X,test_Y=create_dataset(np.array(test_data),100)

In [ ]:
print('train_X:',train_X.shape)
print('validation_X:',val_X.shape)
print('test_X:',test_X.shape)

train_X: (15647, 100, 8)
validation_X: (15647, 100, 8)
test_X: (15647, 100, 8)


In [ ]:
BATCH_SIZE = 512 #16*strategy.num_replicas_in_sync
STEPS_PER_EPOCH = len(train_X) // BATCH_SIZE
VALIDATION_STEP = len(val_X) // BATCH_SIZE
print(BATCH_SIZE,STEPS_PER_EPOCH,VALIDATION_STEP)

512 30 30



#**Model : Wavenet**

In [ ]:
from tensorflow.keras import Model
import tensorflow_probability as tfp
WAVENET_FILTERS = [16, 32]
WAVENET_KERNEL_SIZE = [3, 3]
WAVENET_DILATION_RATE = [8, 5]

In [ ]:
def Correlation(y_true,y_pred): return tf.math.abs(tfp.stats.correlation(y_pred,y_true, sample_axis=None, event_axis=None))

def get_squence_model(x):
    def wave_block(x, filters, kernel_size, n):
        dilation_rates = [2 ** i for i in range(n)]
        x = Conv1D(filters = filters, kernel_size = 1, padding = 'same')(x)
        res_x = x
        for dilation_rate in dilation_rates:
            tanh_out = Conv1D(filters = filters, kernel_size = kernel_size, padding = 'same', activation = 'tanh', dilation_rate = dilation_rate)(x)
            sigm_out = Conv1D(filters = filters, kernel_size = kernel_size, padding = 'same', activation = 'sigmoid', dilation_rate = dilation_rate)(x)
            x = Multiply()([tanh_out, sigm_out])
            x = Conv1D(filters = filters, kernel_size = 1, padding = 'same')(x)
            res_x = Add()([res_x, x])
        return res_x

    for block_idx in range(len(WAVENET_FILTERS)):
        x = wave_block(x, WAVENET_FILTERS[block_idx], WAVENET_KERNEL_SIZE[block_idx], WAVENET_DILATION_RATE[block_idx])         

    return x
def masked_mae(y_true, y_pred):
    mask = tf.math.not_equal(y_true, 0.)
    y_true_masked = tf.boolean_mask(y_true, mask)
    y_pred_masked = tf.boolean_mask(y_pred, mask)
    return tf.keras.losses.mean_absolute_error(y_true = y_true_masked, y_pred = y_pred_masked)

def Model_Wavenet(activations):
    x_input = Input(shape=(100,8,1)) #(100,45,1)
    #branch_outputs = []
    #for i in range(n_assets):
        #a = Lambda(lambda x: x[:,:, i])(x_input) # Slicing the ith asset:
    a = Masking(mask_value = 0., )(x_input)
    a = get_squence_model(a)
    a = Activation(activations[0])(a)
    a = GlobalAveragePooling2D()(a)
    #branch_outputs.append(a)
    #x = Concatenate()(branch_outputs)
    x = Dense(units = 128)(a)
    out = Dense(1)(x)
    model = Model(inputs=x_input, outputs=out)
    return model

In [ ]:
  #checkpoint_path = "/content/model_{}/cp-{epoch:04d}.hd5".format(activations,epoch=0)
  #checkpoint_dir = os.path.dirname(checkpoint_path)
  #checkpoint=ModelCheckpoint(checkpoint_path.format(), monitor='val_mean_squared_error', verbose=0, save_best_only=False)
  #early=EarlyStopping(monitor='val_mean_squared_error', min_delta=0, patience=10, verbose=0, mode='auto')

In [ ]:
def train_and_test_model(activations,shapes,i):
    tf.keras.backend.clear_session()
    test_metrics={}
    #with strategy.scope():
    model=Model_Wavenet(activations) #,optimizer,activations
    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3), loss ='mean_absolute_percentage_error' , metrics=['mean_absolute_percentage_error','mean_squared_error'])
    history=model.fit(x=train_X,y=train_Y,
                      validation_data=(val_X,val_Y),
                      epochs=10,
                      steps_per_epoch=STEPS_PER_EPOCH,
                      validation_steps=VALIDATION_STEP,
                      batch_size=BATCH_SIZE,
                      #use_multiprocessing=True,
                      #workers=4
                      ) #checkpoint,early,  callbacks=[TimeCalcCallback()]      
    history = history.history
    model_eval=model.evaluate(test_X,test_Y,return_dict=True)
    p_test = model.predict(test_X)
    y_test_ = test_Y
    history.update({'activation_functions':len(history['val_mean_absolute_percentage_error'])*[activations],'epochs':list(range(1,len(history['val_mean_absolute_percentage_error'])+1))})
    data_list_test.append([activations,model_eval['mean_absolute_percentage_error'],model_eval['mean_squared_error']])
    del model
    return history

In [ ]:
shapes=[train_X.shape[1],train_X.shape[2]]
activation_strs= ['LeakyReLU','PReLU','sigmoid','softplus','softsign','ReLU',
                  'gelu','selu','elu','tanh','gcu','swish',tfa.activations.mish,
                  'nmcubic','shiftedquadratic','z2cosz','sin','shiftedsinc','dsu','mcubic']
data_list_test=[]
data_list=[]
i=0

In [ ]:
while i<len(activation_strs):
  tf.keras.backend.clear_session()
  print(" Activation Layer : ",activation_strs[i])
  activations=[activation_strs[i]]
  times = []
  history_ = train_and_test_model(activations,shapes,i)
  #history_.update({'time_taken_on_epoch':times})
  data_list.append(pd.DataFrame(history_))
  gc.collect()
  i+=1

 Activation Layer :  LeakyReLU
Epoch 1/10
30/30 [==============================] - 23s 511ms/step - loss: 146.5535 - mean_absolute_percentage_error: 146.5535 - mean_squared_error: 0.0471 - val_loss: 133.1051 - val_mean_absolute_percentage_error: 133.1051 - val_mean_squared_error: 0.1260
Epoch 2/10
30/30 [==============================] - 15s 457ms/step - loss: 87.7530 - mean_absolute_percentage_error: 87.7530 - mean_squared_error: 0.0459 - val_loss: 111.6440 - val_mean_absolute_percentage_error: 111.6440 - val_mean_squared_error: 0.1207
Epoch 3/10
30/30 [==============================] - 13s 452ms/step - loss: 84.3181 - mean_absolute_percentage_error: 84.3181 - mean_squared_error: 0.0460 - val_loss: 108.4876 - val_mean_absolute_percentage_error: 108.4876 - val_mean_squared_error: 0.1199
Epoch 4/10
30/30 [==============================] - 13s 451ms/step - loss: 84.7817 - mean_absolute_percentage_error: 84.7817 - mean_squared_error: 0.0460 - val_loss: 110.4733 - val_mean_absolute_percent

In [ ]:
data = pd.concat(data_list)
data_test = pd.DataFrame(data_list_test)

In [ ]:
data.to_csv('/content/drive/MyDrive/Crypto-research/Train_stats[GRESEARCH-WAVENET].csv',index=False)
data_test.to_csv('/content/drive/MyDrive/Crypto-research/Test_stats[GRESEARCH-WAVENET].csv',index=False)

In [ ]:
data

,loss,mean_absolute_percentage_error,mean_squared_error,val_loss,val_mean_absolute_percentage_error,val_mean_squared_error,activation_functions,epochs
0,147.510681,147.510681,0.195440,143.825348,143.825348,0.218870,[LeakyReLU],1
1,100.413933,100.413933,0.196748,161.102692,161.102692,0.216638,[LeakyReLU],2
2,98.228806,98.228806,0.196844,280.411316,280.411316,0.214495,[LeakyReLU],3
3,95.815582,95.815582,0.197029,190.265228,190.265228,0.216087,[LeakyReLU],4
4,98.313980,98.313980,0.196864,158.010651,158.010651,0.216717,[LeakyReLU],5
...,...,...,...,...,...,...,...,...
5,96.480278,96.480278,0.197022,186.750961,186.750961,0.216217,[mcubic],6
6,97.305893,97.305893,0.196878,97.317131,97.317131,0.218161,[mcubic],7
7,95.546936,95.546936,0.197094,131.208359,131.208359,0.217314,[mcubic],8
8,95.242981,95.242981,0.197082,107.658875,107.658875,0.217833,[mcubic],9


In [ ]:
data_test

,0,1,2
0,[LeakyReLU],110.459335,0.279801
1,[PReLU],101.979691,0.280299
2,[sigmoid],189.090363,0.276016
3,[softplus],132.991989,0.278570
4,[softsign],107.513924,0.281474
5,[ReLU],126.943489,0.278544
6,[gelu],100.305832,0.280457
7,[selu],101.271919,0.280408
8,[elu],130.750031,0.278629
9,[tanh],97.277496,0.280729
